In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append("..")

 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
 
# Import project helpers
from src.data_loading import load_transactions, load_products
from src.preprocessing_transactions import get_transactions_dataset
from src.preprocessing_products import get_products_dataset
from src.evaluation import evaluate_classifier

In [84]:
# Load the transactions data
df_transactions = load_transactions()
df_transactions.info()
df_transactions.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   transaction_id           3000 non-null   object 
 1   customer_id              3000 non-null   object 
 2   transaction_date         3000 non-null   object 
 3   customer_age             3000 non-null   int64  
 4   customer_location        3000 non-null   object 
 5   quantity                 3000 non-null   int64  
 6   unit_price               3000 non-null   float64
 7   total_amount             3000 non-null   float64
 8   payment_method           3000 non-null   object 
 9   shipping_speed           3000 non-null   object 
 10  customer_history_orders  3000 non-null   int64  
 11  discount_applied         3000 non-null   bool   
 12  discount_percentage      3000 non-null   float64
 13  shipping_cost            3000 non-null   float64
 14  delivery_time_days      

,customer_age,quantity,unit_price,total_amount,customer_history_orders,discount_percentage,shipping_cost,delivery_time_days
count,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,48.665667,4.372333,133.781473,431.415573,19.355667,8.573100,14.874780,15.046333
std,17.899889,4.391901,85.403283,331.140587,15.652247,14.689838,5.658677,8.320813
min,18.000000,1.000000,5.200000,6.280000,0.000000,0.000000,5.000000,1.000000
25%,33.000000,2.000000,60.145000,171.685000,4.000000,0.000000,9.950000,8.000000
50%,48.000000,3.000000,112.465000,322.585000,18.000000,0.000000,14.780000,15.000000
75%,64.000000,4.000000,210.372500,623.160000,33.000000,13.700000,19.765000,22.000000
max,79.000000,19.000000,299.590000,1865.660000,49.000000,50.000000,24.990000,29.000000


In [85]:
# Plotting

In [86]:
X_train, X_test, y_train, y_test, preprocessor = get_transactions_dataset()

# Define the Logistic Regression classifier (linear model for binary classification)
logreg = LogisticRegression(
    max_iter=1000,   # allow more iterations so the optimizer can converge
    random_state=42, # make results reproducible
    n_jobs=-1,       # use all available CPU cores (for solvers that support it)
    class_weight="balanced",

#For LogReg and SVM, you now see:
#Recall = 1.0 → they catch all counterfeit transactions in the test set
#Precision dipped a bit (~0.967) → they flag a few more false positives
#That’s exactly what “balanced” does: it makes the model care more about the minority class, pushing for higher recall.
)

# Build a Pipeline:
# 1) "preprocess" step: applies scaling + one-hot encoding to features
# 2) "model" step: fits Logistic Regression on the transformed features
clf = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", logreg),
])

# Train the pipeline on the training data
clf.fit(X_train, y_train)

# Evaluate the trained model on the test data
evaluate_classifier(
    clf,
    X_test,
    y_test,
    model_name="logreg",
    dataset_name="transactions",
)


=== logreg on transactions ===
Accuracy : 0.9917
Precision: 0.9671
Recall   : 1.0000
F1-score : 0.9833
ROC-AUC  : 0.9999


In [87]:
# Plotting

In [88]:
# Define the KNN classifier
knn = KNeighborsClassifier(
    n_neighbors=5,     # look at 5 nearest neighbors
    weights="distance",# closer neighbors have more influence
    n_jobs=-1,         # use all CPU cores (for distance computations)
)

# Pipeline: preprocessing -> KNN model
clf = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", knn),
])

# Train the model
clf.fit(X_train, y_train)

# Evaluate on the test set
evaluate_classifier(
    clf,
    X_test,
    y_test,
    model_name="knn",
    dataset_name="transactions",
)


=== knn on transactions ===
Accuracy : 0.9783
Precision: 0.9653
Recall   : 0.9456
F1-score : 0.9553
ROC-AUC  : 0.9982


In [89]:
# Plotting

In [90]:
# Define the Gradient Boosting classifier
# (sequentially builds an ensemble of small decision trees)
gb = GradientBoostingClassifier(
    random_state=42,  # for reproducibility
)

# Pipeline: preprocessing -> gradient boosting model
clf = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", gb),
])

# Train the model
clf.fit(X_train, y_train)

# Evaluate on the test set
evaluate_classifier(
    clf,
    X_test,
    y_test,
    model_name="gb",
    dataset_name="transactions",
)


=== gb on transactions ===
Accuracy : 0.9950
Precision: 0.9800
Recall   : 1.0000
F1-score : 0.9899
ROC-AUC  : 0.9999


In [ ]:
# Plotting

In [91]:
# Define the Random Forest model (ensemble of decision trees)
rf = RandomForestClassifier(
    n_estimators=200,   # number of trees in the forest
    max_depth=None,     # trees can grow until leaves are pure or min_samples reached
    random_state=42,    # reproducible results
    n_jobs=-1,          # use all CPU cores
    class_weight="balanced",
)

# Build a Pipeline: first apply preprocessing, then fit the RF model
clf = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", rf),
])

# Fit the pipeline on training data (preprocessing is learned + model is trained)
clf.fit(X_train, y_train)

# Evaluate on test data and print/save metrics + plots
evaluate_classifier(
    clf,
    X_test,
    y_test,
    model_name="rf",
    dataset_name="transactions",
)


=== rf on transactions ===
Accuracy : 0.9950
Precision: 0.9932
Recall   : 0.9864
F1-score : 0.9898
ROC-AUC  : 0.9999


In [92]:
# Plotting

In [93]:
# Define an SVM classifier with an RBF kernel
svm = SVC(
    kernel="rbf",       # non-linear kernel (Radial Basis Function)
    probability=True,   # enable probability estimates for ROC-AUC
    random_state=42,    # reproducibility
    class_weight="balanced",
)

# Pipeline: preprocessing -> SVM model
clf = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", svm),
])

# Train the model
clf.fit(X_train, y_train)

# Evaluate on the test set
evaluate_classifier(
    clf,
    X_test,
    y_test,
    model_name="svm",
    dataset_name="transactions",
)


=== svm on transactions ===
Accuracy : 0.9917
Precision: 0.9671
Recall   : 1.0000
F1-score : 0.9833
ROC-AUC  : 0.9996


In [ ]:
# Plotting